In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras.utils import to_categorical

In [2]:
def pre_processamento():
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_images = train_images.reshape((60000,28,28,1))
    train_images = train_images.astype('float32')/255

    test_images = test_images.reshape((10000,28,28,1))
    test_images = test_images.astype('float32')/255

    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)

    return train_images, train_labels, test_images, test_labels

In [3]:
# Definir a arquitetura da CNN
def criar_modelo(conv_layers, filters, dense_size):
    model = models.Sequential()

    # Adicionar camadas de convolução-pooling
    for i in range(conv_layers):
        model.add(layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape = (28,28,1)))
        print("Conv2D")
        if i != conv_layers - 1 or conv_layers == 1: #se não for a última camada de conv, faz MaxPooling2D, com exceção se tiver somente 1 camada de conv, nesse caso faz MaxPooling2D (mesmo ela sendo a última também)
            print("MaxPooling2D")
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(dense_size, activation='relu'))
    model.add(layers.Dense(10, activation='softmax')) # 10 é o número de classes
    
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [4]:
train_images, train_labels, test_images, test_labels = pre_processamento()

In [5]:
# Definir os valores fixos dos parâmetros
filters = 32
dense_size = 64

# Testar diferentes quantidades de camadas de convolução-pooling
conv_layers_list = [1, 2, 3]

In [6]:
melhor_acc = 0
melhor_layer = 0
for layer in conv_layers_list:
    model = criar_modelo(layer, filters, dense_size)
    model.fit(train_images, train_labels, epochs=5, batch_size = 64)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_loss:", test_loss, "\ntest_acc:", test_acc)
    print(f"Essa acurácia significa que o modelo usando layer: {layer}, filter: {filters} e tamanho da camada densa: {dense_size} é  capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")
    if test_acc > melhor_acc:
        melhor_acc = test_acc
        melhor_layer = layer
print(f"Portanto, a melhor layer é a {melhor_layer}, que possui {round(melhor_acc*100, 1)} de acurácia.")

Conv2D
MaxPooling2D
Epoch 1/5
938/938 [==============================] - 25s 25ms/step - loss: 0.4640 - accuracy: 0.8361
Epoch 2/5
938/938 [==============================] - 25s 27ms/step - loss: 0.2935 - accuracy: 0.8957
Epoch 3/5
938/938 [==============================] - 21s 22ms/step - loss: 0.2522 - accuracy: 0.9081
Epoch 4/5
938/938 [==============================] - 23s 25ms/step - loss: 0.2237 - accuracy: 0.9179
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.2802 - accuracy: 0.9057
test_loss: 0.28022149205207825 
test_acc: 0.9057000279426575
Essa acurácia significa que o modelo usando layer: 1, filter: 32 e tamanho da camada densa: 64 é  capaz de classificar corretamente 90.6%  das imagens
Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 41s 43ms/step - loss: 0.4398 - accuracy: 0.8428
Epoch 2/5
938/938 [==============================] - 36s 38ms/step - loss: 0.2768 - accuracy: 0.8992
Epoch 3/5
938/938 [=================

In [7]:
filters_list = [16, 32, 64]

In [8]:
melhor_acc = 0
melhor_filter = 0
for filters in filters_list:
    model = criar_modelo(melhor_layer, filters, dense_size)
    model.fit(train_images, train_labels, epochs=5, batch_size = 64)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_loss:", test_loss, "\ntest_acc:", test_acc)
    print(f"Essa acurácia significa que o modelo usando layer: {melhor_layer}, filter: {filters} e tamanho da camada densa: {dense_size} é capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")
    if test_acc > melhor_acc:
        melhor_acc = test_acc
        melhor_filter = filters
print(f"Portanto, o melhor filter é o {melhor_filter}, que possui {round(melhor_acc*100, 1)} de acurácia.")

Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 20s 21ms/step - loss: 0.4651 - accuracy: 0.8326
Epoch 2/5
938/938 [==============================] - 18s 19ms/step - loss: 0.3093 - accuracy: 0.8877
Epoch 3/5
938/938 [==============================] - 18s 19ms/step - loss: 0.2594 - accuracy: 0.9043
Epoch 4/5
938/938 [==============================] - 18s 19ms/step - loss: 0.2282 - accuracy: 0.9163
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.2746 - accuracy: 0.9010
test_loss: 0.2745843231678009 
test_acc: 0.9010000228881836
Essa acurácia significa que o modelo usando layer: 2, filter: 16 e tamanho da camada densa: 64 é capaz de classificar corretamente 90.1%  das imagens
Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 35s 36ms/step - loss: 0.4488 - accuracy: 0.8404
Epoch 2/5
938/938 [==============================] - 34s 36ms/step - loss: 0.2853 - accuracy: 0.8967
Epoch 3/5
938/938 [============

In [9]:
dense_size_list = [64, 128, 256]

In [10]:
melhor_acc = 0
melhor_dense = 0
for dense_size in dense_size_list:
    model = criar_modelo(melhor_layer, melhor_filter, dense_size)
    model.fit(train_images, train_labels, epochs=5, batch_size = 64)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_loss:", test_loss, "\ntest_acc:", test_acc)
    print(f"Essa acurácia significa que o modelo usando layer: {melhor_layer}, filter: {melhor_filter} e tamanho da camada densa: {dense_size} é capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")
    if test_acc > melhor_acc:
        melhor_acc = test_acc
        melhor_dense = dense_size
print(f"Portanto, o melhor tamanho da camada densa é {melhor_dense}, que possui {round(melhor_acc*100, 1)} de acurácia.")

Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 71s 74ms/step - loss: 0.4138 - accuracy: 0.8520
Epoch 2/5
938/938 [==============================] - 69s 74ms/step - loss: 0.2555 - accuracy: 0.9075
Epoch 3/5
938/938 [==============================] - 72s 76ms/step - loss: 0.2086 - accuracy: 0.9242
Epoch 4/5
938/938 [==============================] - 73s 78ms/step - loss: 0.1767 - accuracy: 0.9347
Epoch 5/5
313/313 [==============================] - 4s 12ms/step - loss: 0.2348 - accuracy: 0.9169
test_loss: 0.2348276823759079 
test_acc: 0.9168999791145325
Essa acurácia significa que o modelo usando layer: 2, filter: 64 e tamanho da camada densa: 64 é capaz de classificar corretamente 91.7%  das imagens
Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 80s 85ms/step - loss: 0.4004 - accuracy: 0.8547
Epoch 2/5
938/938 [==============================] - 75s 80ms/step - loss: 0.2456 - accuracy: 0.9092
Epoch 3/5
938/938 [===========

In [11]:
print(f"Ao final, a melhor combinação foi: \nlayer: {melhor_layer}, filter: {melhor_filter} e tamanho da camada densa: {melhor_dense}")

Ao final, a melhor combinação foi: 
layer: 2, filter: 64 e tamanho da camada densa: 256


In [12]:
# 0.9083999991416931 same

# 90.7 valid

# 91.1 valid input

# 91.7 same input

# 91.2 same 

O que é dropout? Avalie diferentes porcentagens de dropout. Quanto dropout é melhor?

O Dropout é uma técnica de regularização utilizada para reduzir o overfitting em redes neurais. Durante o treinamento, uma proporção dos neurônios é aleatoriamente "desligada" (dropout) em cada atualização do gradiente, o que força a rede a aprender recursos mais robustos e evita a dependência excessiva de neurônios específicos.

Vamos modificar a função create_cnn_model_with_dense_size para adicionar uma camada Dropout antes da camada densa:

In [8]:
# Definir a arquitetura da CNN
def criar_modelo_com_dropout(conv_layers, filters, dense_size, dropout_rate):
    model = models.Sequential()

    # Adicionar camadas de convolução-pooling
    for i in range(conv_layers):
        model.add(layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape = (28,28,1)))
        print("Conv2D")
        if i != conv_layers - 1 or conv_layers == 1: #se não for a última camada de conv, faz MaxPooling2D, com exceção se tiver somente 1 camada de conv, nesse caso faz MaxPooling2D (mesmo ela sendo a última também)
            print("MaxPooling2D")
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(dense_size, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation='softmax')) # 10 é o número de classes
    
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [14]:
dropout_rates = [0.1, 0.3, 0.5, 0.7]

melhor_acc = 0
melhor_dropout = 0
for dropout_rate in dropout_rates:
    model = criar_modelo_com_dropout(melhor_layer, melhor_filter, melhor_dense, dropout_rate)
    model.fit(train_images, train_labels, epochs=5, batch_size = 64)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_loss:", test_loss, "\ntest_acc:", test_acc)
    print(f"Essa acurácia significa que o modelo usando layer: {melhor_layer}, filter: {melhor_filter}, tamanho da camada densa: {melhor_dense} e dropout: {dropout_rate} é capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")
    if test_acc > melhor_acc:
        melhor_acc = test_acc
        melhor_dropout = dropout_rate
print(f"Portanto, o melhor dropout é o {melhor_dropout}, que possui {round(melhor_acc*100, 1)} de acurácia.")

Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 80s 85ms/step - loss: 0.4003 - accuracy: 0.8554
Epoch 2/5
938/938 [==============================] - 79s 85ms/step - loss: 0.2439 - accuracy: 0.9103
Epoch 3/5
938/938 [==============================] - 80s 85ms/step - loss: 0.1977 - accuracy: 0.9269
Epoch 4/5
938/938 [==============================] - 80s 85ms/step - loss: 0.1643 - accuracy: 0.9391
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 0.2513 - accuracy: 0.9201
test_loss: 0.2513017952442169 
test_acc: 0.9200999736785889
Essa acurácia significa que o modelo usando layer: 2, filter: 64, tamanho da camada densa: 256 e dropout: 0.1 é capaz de classificar corretamente 92.0%  das imagens
Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 80s 85ms/step - loss: 0.4155 - accuracy: 0.8513
Epoch 2/5
938/938 [==============================] - 79s 85ms/step - loss: 0.2578 - accuracy: 0.9054
Epoch 3/5
938/9

E. O que é batch normalization? Aplique batch normalization e avalie como ela ajuda?

Batch Normalization é uma técnica usada para acelerar o treinamento de redes neurais e estabilizar o processo de aprendizado. Ela normaliza a ativação de cada camada, aplicando uma transformação que mantém a média próxima de zero e o desvio padrão próximo de um. Isso ajuda a reduzir a covariância de ativação entre as camadas e torna o treinamento mais rápido e estável.

Vamos modificar a função create_cnn_model_with_dense_dropout para adicionar uma camada Batch Normalization antes da camada densa:

In [9]:
# Definir a arquitetura da CNN
def criar_modelo_com_batchnorm(conv_layers, filters, dense_size, dropout_rate):
    model = models.Sequential()

    # Adicionar camadas de convolução-pooling
    for i in range(conv_layers):
        model.add(layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape = (28,28,1)))
        print("Conv2D")
        if i != conv_layers - 1 or conv_layers == 1: #se não for a última camada de conv, faz MaxPooling2D, com exceção se tiver somente 1 camada de conv, nesse caso faz MaxPooling2D (mesmo ela sendo a última também)
            print("MaxPooling2D")
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(dense_size, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation='softmax')) # 10 é o número de classes
    
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [16]:
model = criar_modelo_com_batchnorm(melhor_layer, melhor_filter, melhor_dense, melhor_dropout)
model.fit(train_images, train_labels, epochs=5, batch_size = 64)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test_loss:", test_loss, "\ntest_acc:", test_acc)
print(f"Essa acurácia significa que o modelo usando layer: {melhor_layer}, filter: {melhor_filter}, tamanho da camada densa: {melhor_dense}, dropout: {melhor_dropout} e com batch normalization é capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")

print(f"Portanto, possui {round(test_acc*100, 1)} de acurácia.")

Conv2D
MaxPooling2D
Conv2D
Epoch 1/5
938/938 [==============================] - 99s 105ms/step - loss: 0.3500 - accuracy: 0.8754
Epoch 2/5
938/938 [==============================] - 105s 112ms/step - loss: 0.2348 - accuracy: 0.9149
Epoch 3/5
938/938 [==============================] - 100s 106ms/step - loss: 0.1958 - accuracy: 0.9281
Epoch 4/5
938/938 [==============================] - 96s 102ms/step - loss: 0.1668 - accuracy: 0.9392
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 0.2380 - accuracy: 0.9180
test_loss: 0.2379808872938156 
test_acc: 0.9179999828338623
Essa acurácia significa que o modelo usando layer: 2, filter: 64, tamanho da camada densa: 256, dropout: 0.1 e com batch normalization é capaz de classificar corretamente 91.8%  das imagens
Portanto, possui 91.8 de acurácia.


F. O que é data augmentation? Aplique data augmentation e avalie como ele ajuda?

Data Augmentation é uma técnica usada para expandir o conjunto de dados de treinamento, aplicando transformações aleatórias nos dados existentes, como rotação, zoom, espelhamento, deslocamento, entre outros. Essa técnica é útil quando o conjunto de dados de treinamento é limitado, pois permite aumentar a diversidade dos exemplos apresentados ao modelo.

Vamos usar a biblioteca imgaug para aplicar Data Augmentation no conjunto de dados Fashion MNIST. Primeiro, instale a biblioteca usando o seguinte comando: pip install imgaug

In [14]:
import imgaug.augmenters as iaa

# Definir as transformações de data augmentation
augmenter = iaa.Sequential([
    # iaa.Fliplr(0.5),  # Espelhamento horizontal com probabilidade de 0.5
    iaa.Crop(percent=(0, 0.1)),  # Corte aleatório de até 10% da imagem
    iaa.GaussianBlur(sigma=(0, 1.0)),  # Desfoque Gaussiano com desvio padrão entre 0 e 1.0
    iaa.Affine(rotate=(-20, 20)),  # Rotação aleatória entre -20 e 20 graus
    iaa.AddToHueAndSaturation(value=(-10, 10))  # Alteração aleatória no valor da saturação
])

In [15]:
model = criar_modelo_com_batchnorm(2, 64, 256, 0.1)


Conv2D
MaxPooling2D
Conv2D


In [19]:
# Converter as imagens para uint8 e expandir a dimensão para três canais
train_images_uint8 = np.expand_dims((train_images * 255).astype(np.uint8), axis=-1)
train_images_uint8 = np.repeat(train_images_uint8, 3, axis=-1)

# Aplicar Data Augmentation nos dados de treinamento
train_images_augmented = augmenter(images=train_images_uint8)


# Concatenar os dados originais com os dados aumentados
train_images_combined = np.concatenate([train_images_uint8, train_images_augmented])
train_labels_combined = np.concatenate([train_labels, train_labels])

# Treinar o modelo com os dados aumentados
model.fit(train_images_combined, train_labels_combined, epochs=5, batch_size=64, verbose=0)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test_loss:", test_loss, "\ntest_acc:", test_acc)
print(f"Essa acurácia significa que o modelo usando layer: {melhor_layer}, filter: {melhor_filter}, tamanho da camada densa: {melhor_dense}, dropout: {dropout_rate}, com batch normalization e com data augmentation é capaz de classificar corretamente {round(test_acc*100, 1)}%  das imagens")

print(f"Portanto, possui {round(test_acc*100, 1)} de acurácia.")

AssertionError: Expected image with two or three dimensions, but got 4 dimensions and shape (23, 26, 1, 3).